# Load and process GBD data 

The dataset can be downloaded via a [permalink](http://ghdx.healthdata.org/gbd-results-tool?params=gbd-api-2019-permalink/c28e0baacd911b17c2f5ea1af0aca42e).

The output is a csv file, this can be loaded as is into the python code.
The Country names are converted to fit with the standard used in this code (see last cell for details).
Some Countries are removed as there isn't data available regarding population or urbanization.

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.optimize import curve_fit
from lmfit.models import StepModel, LinearModel


#dictionary mapping gdb country names to the names used in our plots
rename_gbd_data = {
"Bolivia (Plurinational State of)" : "Bolivia",
"Brunei Darussalam":	"Brunei",
"Côte d'Ivoire" : "Cote d'Ivoire",
"Cabo Verde": "Cape Verde",
"Congo":"Congo, Rep.",
"Czechia":	"Czech Republic",
"Democratic People's Republic of Korea" : "North Korea",
"Iran (Islamic Republic of)" : "Iran",
"Lao People's Democratic Republic" : "Lao",
"Micronesia (Federated States of)" : "Micronesia, Fed. Sts.",
"Republic of Moldova" : "Moldova",
"Republic of Korea":"South Korea",
"Russian Federation":"Russia",
"Syrian Arab Republic":"Syria"
}

#countries in gdb not in other data sets - remove from dataframe, Eritrea and Zimbabwe are removed because they have no urbanization data
remove_from_gdb = [
"American Samoa",
"Bermuda",
"Cook Islands",
"Greenland",
"Guam",
"Holy See",
"Niue",
"Northern Mariana Islands",
"Puerto Rico",
"Taiwan (Province of China)",
"Tokelau",
"United States Virgin Islands",
"Eritrea", 
"Zimbabwe" 
]

#Read in Global Burden of Disease data
df = pd.read_csv('data/gbd-prevelance-rate-2017.csv')

#Remove countries in GDB not in other datasets
gbd_df = df[~df['location_name'].isin(remove_from_gdb)]

#Replace gbd country names with correct ones to be consistent
gbd_df["location_name"].replace(rename_gbd_data, inplace=True)

#Throw away columns we don't use
gbd_df.drop(['measure_id', 'measure_name', 'location_id','sex_id', 'sex_name', 'age_id', 'age_name', 'cause_id', 'metric_id'], axis=1, inplace=True)

#Create separate dataframes for each disorder
#The appropriate suffix is added to each column name to keep the unqiue. We then rename key column (location_name) to be same for merge later
anxietydf = gbd_df.loc[(gbd_df['cause_name'] == 'Anxiety disorders') & (gbd_df['metric_name'] == 'Rate')].add_suffix('_anxiety')
anxietydf.rename(columns={'location_name_anxiety': 'location_name'}, inplace = True)

depressivedf = gbd_df.loc[(gbd_df['cause_name'] == 'Depressive disorders') & (gbd_df['metric_name'] == 'Rate')].add_suffix('_depressive')
depressivedf.rename(columns={'location_name_depressive': 'location_name'}, inplace = True)

substanceusedf = gbd_df.loc[(gbd_df['cause_name'] == 'Substance use disorders') & (gbd_df['metric_name'] == 'Rate')].add_suffix('_substanceuse')
substanceusedf.rename(columns={'location_name_substanceuse': 'location_name'}, inplace = True)

# Load and Process Urban data 

The dataset can be downloaded via a [Gapmider](https://www.gapminder.org/data/).

Two files are downloaded: 

1. Urban Population as % Total
2. Population Total (see [here](http://gapm.io/dpop))

Note these files include various years - the final dataframe includes all years that are common in both files.

The Country names are converted to fit with the standard used in this code (see last cell for details).
Some Countries are removed as there isn't data available regarding population or urbanization or GBD.

In [2]:
#Rename country names to plot standards
rename_urban_data = {
"Congo, Dem. Rep.": "Democratic Republic of the Congo",
"Kyrgyz Republic":	"Kyrgyzstan",
"St. Kitts and Nevis" : "Saint Kitts and Nevis",
"St. Lucia" : "Saint Lucia",
"St. Vincent and the Grenadines" : "Saint Vincent and the Grenadines",
"Slovak Republic":"Slovakia",
"Tanzania":"United Republic of Tanzania",
"United States": "United States of America",
"Venezuela":"Venezuela (Bolivarian Republic of)",
"Vietnam":"Viet Nam"
}

#Removes countries - some are not in GBD data, and some have no data for urbanisation
remove_from_urban = ["Liechtenstein","Eritrea", "Holy See", "Zimbabwe" ]

#Read in population and urbanization data
urban_df = pd.read_csv('data/urban_population_percent.csv')
pop_df = pd.read_csv('data/population_total.csv')

#Find the years contained within both datasets - 1960-2019
#NOTE if you want all possible years uncomment this line and comment drop section below
#cols_to_use = urban_df.columns.intersection(pop_df.columns)

#Only keep 2015-2019 for now - makes frames smaller and more manageable
colstokeep = ['country','2015','2016','2017','2018','2019']
colstodrop_pop = list(set(pop_df.columns)-set(colstokeep)) 
colstodrop_urban = list(set(urban_df.columns)-set(colstokeep)) 
urban_df.drop(colstodrop_urban, axis=1,inplace=True)
pop_df.drop(colstodrop_pop, axis=1,inplace=True)

#Merge the two data frames giving suffixes for urbanization and population
urbanMerged = urban_df.merge(pop_df, on='country',how='outer', suffixes=('_urban', '_pop'))
u2 = urbanMerged

#Remove countries in this data not in other datasets
urbanMerged = urbanMerged[~urbanMerged['country'].isin(remove_from_urban)]

#Replace Urbanisation country names with correct ones to be consistent
urbanMerged["country"].replace(rename_urban_data, inplace=True)

#Rename country column to location_name to be consistent with GBD
urbanMerged = urbanMerged.rename(columns={'country': 'location_name'})


# Create final data frame

For now we merge the urban dataframe (containing population and urbanization) with the 3 disorder dataframes one by one.



In [3]:
fullMergeda = urbanMerged.merge(anxietydf, on='location_name',how='outer')
fullMergedb = fullMergeda.merge(depressivedf, on='location_name',how='outer')
df = fullMergedb.merge(substanceusedf, on='location_name',how='outer') #our final dataframe

#Export the final data frame to csv for ref
df.to_csv("data/plot_data.csv", sep='\t', encoding='utf-8')


# Define functions and colors and read in files for plots

In [7]:
import numpy as np
import pylab as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import datetime
import scipy.interpolate
import math

def sample_lowess(x, y, xgrid, sample_size, frac, it):
    #Pick some random indecies from our data to sample for the lowess fit
    samples = np.random.choice(len(x), sample_size, replace=True)
    #Create a new lowess using on sample of original data
    y_sm = sm.nonparametric.lowess(y[samples],x[samples], frac, it,return_sorted = False)
    # regularly sample it onto the specified grid
    y_grid = scipy.interpolate.interp1d(x[samples], y_sm, fill_value='extrapolate')(xgrid)
    return y_grid


def generateBootstrap(x,y,frac, it):
    xgrid = np.linspace(x.min(),x.max(), num =100) #Define x resolution of interpolation
    number_samples = 500 #number of lowess fits to sample
    sample_size = math.floor(0.5 * len(x)) #sample size used to generate fit - use 50%
    bs_fits = np.stack([sample_lowess(x, y, xgrid, sample_size, frac, it) for k in range(number_samples)]).T
    mean = np.nanmean(bs_fits, axis=1) #mean of fits
    stderr = np.nanstd(bs_fits, axis=1, ddof=0) #stddev of fits
    return xgrid, mean, stderr

#Colors for different data - use safe template
colors = px.colors.qualitative.Safe[:3]
transparentcolors = []
for i in px.colors.qualitative.Safe[:3]:
        transparentcolors.append(('rgba' + i[3:-1] + ',0.25)'))
colors = [colors, transparentcolors]

# plotly figure setup
def plotdata(x,y,labels,colors,h, bubble=False, msize=[]):
    sm_x = [None] * 3
    sm_y = [None] * 3
    xgrid = [None] * 3
    mean = [None] * 3
    stderr = [None] * 3

    #Parameters for lowess fit - using default values
    frac = 2./3.
    it = 3
    
    #Generate trendlines and bootstrap confidence intervals
    for i in range(len(y)):
        sm_x[i], sm_y[i] = sm.nonparametric.lowess(y[i], x,  frac, it, return_sorted = True).T
        xgrid[i], mean[i], stderr[i] = generateBootstrap(x, y[i], frac, it)

    fig=go.Figure()
    if bubble:
        max_size = 60
        sizeref = msize.max() / max_size ** 2
        # plotly figure setup
        for i in range(len(y)):    
            fig.add_trace(go.Scatter(name=labels[i], x=x, y=y[i].values, mode='markers', hovertext=h, 
                                    marker=dict(color=colors[0][i], size = msize, sizeref=sizeref, sizemin=5, sizemode="area", line=dict(color='gray',width=1))))
    else:
        for i in range(len(y)):    
            fig.add_trace(go.Scatter(name=labels[i], x=x, y=y[i].values, mode='markers', hovertext=h, marker=dict(color=colors[0][i], 
                                                                                                                   size = 12, 
                                                                                                                   line=dict(color='gray', width=1))))
    #plot lowess fits and errors
    for i in range(len(y)):
        fig.add_trace(go.Scatter(name='Lowess Fit '+labels[i], x=xgrid[i], y=mean[i], mode='lines',line=dict(color=colors[0][i])))
        fig.add_trace(go.Scatter(
            name='Upper Bound',
            x=xgrid[i],
            y=mean[i]+1.96*stderr[i],
            mode='lines',
            marker=dict(color=colors[0][i]),
            line=dict(width=0),
            showlegend=False
        ))
        fig.add_trace(go.Scatter(
            name='Lower Bound',
            x=xgrid[i],
            y=mean[i]-1.96*stderr[i],
            marker=dict(color=colors[0][i]),
            line=dict(width=0),
            mode='lines',
            fillcolor=colors[1][i],
            fill='tonexty',
            showlegend=False
        ))
    fig.update_layout(
        autosize=True,
        width=2000,
        height=1600,
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4
        ),
        font=dict(size=24),
        legend=dict(
            x=0.05,
            y=0.95,
            traceorder='normal',
            font=dict(
                size=24),
        ),
        xaxis_title_text='Urbanization Fraction',
        yaxis_title_text='Rate in Population',
        template='ggplot2' # choose from one of the pre-defined templates

    )
    return fig 


# Plot CMDs vs urban population with lowess fit and CI

In [5]:
#Column headers and x,y values
labels = ['Anxiety Disorders', 'Depressive Disorders','Substance Use Disorders']

x = df['2017_urban'] #2017 urbanization data
y = [df['val_anxiety'].div(100000), df['val_depressive'].div(100000), df['val_substanceuse'].div(100000)]
hovertext = df['location_name']

fig = plotdata(x,y,labels, colors, hovertext, bubble=False)
fig.show()
cmd_lowess_plot = fig

# Bubble plot scaled by population size

In [8]:
#Column headers and x,y values
labels = ['Anxiety Disorders', 'Depressive Disorders','Substance Use Disorders']
x = df['2017_urban'] #2017 urbanization data
y = [df['val_anxiety'].div(100000), df['val_depressive'].div(100000), df['val_substanceuse'].div(100000)]
hovertext = df['location_name']

#Generate plot and save for later
fig = plotdata(x,y,labels, colors, hovertext, bubble=True, msize=df['2017_pop'])
fig.show()
bubble_plot= fig


# Plots with Ordinary Least Square (OLS) fit

In [9]:
def plotlinearfit(x,y,labels,colors, bubble=False, size=[]):
    fig=go.Figure()
    for i in range(len(y)):    
            fig.add_trace(go.Scatter(name=labels[i], x=x, y=y[i].values, mode='markers', marker=dict(color=colors[0][i], size = 12, line=dict(color='gray',width=1))))
    for i in range(len(y)):
        fit = sm.OLS(y[i],sm.add_constant(x)).fit()
        print(fit.summary())
        fig.add_trace(go.Scatter(name='OLS fit '+labels[i], x=x, y=fit.fittedvalues, mode='lines',line=dict(color=colors[0][i])))

    fig.update_layout(
        autosize=True,
        width=2000,
        height=1600,
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4
        ),
        font=dict(size=24),
        legend=dict(
            x=0.05,
            y=0.95,
            traceorder='normal',
            font=dict(
                size=24),
        ),
        xaxis_title_text='Urbanization Fraction',
        yaxis_title_text='Rate in Population',
        template='ggplot2' # choose from one of the pre-defined templates
    )
    return fig 

labels = ['Anxiety Disorders', 'Depressive Disorders','Substance Use Disorders']

x = df['2017_urban'] #2017 urbanization data
y = [df['val_anxiety'].div(100000), df['val_depressive'].div(100000), df['val_substanceuse'].div(100000)]
hovertext = df['location_name']

fig = plotlinearfit(x,y,labels, colors, bubble=False)
fig.show()
cmd_linear_plot = fig

                            OLS Regression Results                            
Dep. Variable:            val_anxiety   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.214
Method:                 Least Squares   F-statistic:                     52.85
Date:                Sat, 23 Jan 2021   Prob (F-statistic):           9.24e-12
Time:                        01:12:18   Log-Likelihood:                 592.92
No. Observations:                 191   AIC:                            -1182.
Df Residuals:                     189   BIC:                            -1175.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0274      0.002     12.595      0.0

# Plot estimate error from GBD data vs urbanization

In [10]:
#Column headers and x,y values
labels = ['Anxiety Error', 'Depressive Error','Substance Use Error']
x = df['2017_urban']
anx_error = (df['upper_anxiety']-df['lower_anxiety'])/df['val_anxiety']
dep_error = (df['upper_depressive']-df['lower_depressive'])/df['val_depressive']
sub_error = (df['upper_substanceuse']-df['lower_substanceuse'])/df['val_substanceuse']
y = [anx_error, dep_error, sub_error]
hovertext = df['location_name']

fig = plotdata(x,y,labels, colors, hovertext, bubble=False)
fig.update_layout(
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder='normal',
        font=dict(
            size=24),
    ),
    yaxis_range=[0.1,0.65],
    xaxis_title_text='Urbanization Fraction',
    yaxis_title_text='Estimate error as fraction of mean',
    template='ggplot2' # choose from one of the pre-defined templates
)
fig.show()
error_plot= fig

# Post plots online and export PDFs

In [11]:
import chart_studio.plotly as py
import chart_studio.tools as cst

py.plot(cmd_lowess_plot, filename = 'cmd_lowess_plot', auto_open=False)
py.plot(bubble_plot, filename  = 'bubble_plot', auto_open=False)
py.plot(error_plot, filename = 'error_plot', auto_open=False)
py.plot(cmd_linear_plot, filename = 'cmd_linear_plot', auto_open=False)


cmd_lowess_plot.write_image("images/lowess.pdf", width=2000, height=1600)
bubble_plot.write_image("images/bubble.pdf",width=2000, height=1600)
error_plot.write_image("images/error.pdf",width=2000, height=1600)
cmd_linear_plot.write_image("images/linear.pdf",width=2000, height=1600)

In [ ]:
#country name standard used in dataframes
namesinplots = ["Afghanistan",
"Albania",
"Algeria",
"Andorra",
"Angola",
"Antigua and Barbuda",
"Argentina",
"Armenia",
"Australia",
"Austria",
"Azerbaijan",
"Bahamas",
"Bahrain",
"Bangladesh",
"Barbados",
"Belarus",
"Belgium",
"Belize",
"Benin",
"Bhutan",
"Bolivia",
"Bosnia and Herzegovina",
"Botswana",
"Brazil",
"Brunei",
"Bulgaria",
"Burkina Faso",
"Burundi",
"Cambodia",
"Cameroon",
"Canada",
"Cape Verde",
"Central African Republic",
"Chad",
"Chile",
"China",
"Colombia",
"Comoros",
"Congo, Rep.",
"Costa Rica",
"Cote d'Ivoire",
"Croatia",
"Cuba",
"Cyprus",
"Czech Republic",
"Democratic Republic of the Congo",
"Denmark",
"Djibouti",
"Dominica",
"Dominican Republic",
"Ecuador",
"Egypt",
"El Salvador",
"Equatorial Guinea",
"Eritrea",
"Estonia",
"Eswatini",
"Ethiopia",
"Fiji",
"Finland",
"France",
"Gabon",
"Gambia",
"Georgia",
"Germany",
"Ghana",
"Greece",
"Grenada",
"Guatemala",
"Guinea",
"Guinea-Bissau",
"Guyana",
"Haiti",
"Honduras",
"Hungary",
"Iceland",
"India",
"Indonesia",
"Iran",
"Iraq",
"Ireland",
"Israel",
"Italy",
"Jamaica",
"Japan",
"Jordan",
"Kazakhstan",
"Kenya",
"Kiribati",
"Kuwait",
"Kyrgyzstan",
"Lao",
"Latvia",
"Lebanon",
"Lesotho",
"Liberia",
"Libya",
"Lithuania",
"Luxembourg",
"Madagascar",
"Malawi",
"Malaysia",
"Maldives",
"Mali",
"Malta",
"Marshall Islands",
"Mauritania",
"Mauritius",
"Mexico",
"Micronesia, Fed. Sts.",
"Moldova",
"Monaco",
"Mongolia",
"Montenegro",
"Morocco",
"Mozambique",
"Myanmar",
"Namibia",
"Nauru",
"Nepal",
"Netherlands",
"New Zealand",
"Nicaragua",
"Niger",
"Nigeria",
"North Korea",
"North Macedonia",
"Norway",
"Oman",
"Pakistan",
"Palau",
"Palestine",
"Panama",
"Papua New Guinea",
"Paraguay",
"Peru",
"Philippines",
"Poland",
"Portugal",
"Qatar",
"Romania",
"Russia",
"Rwanda",
"Saint Kitts and Nevis",
"Saint Lucia",
"Saint Vincent and the Grenadines",
"Samoa",
"San Marino",
"Sao Tome and Principe",
"Saudi Arabia",
"Senegal",
"Serbia",
"Seychelles",
"Sierra Leone",
"Singapore",
"Slovakia",
"Slovenia",
"Solomon Islands",
"Somalia",
"South Africa",
"South Korea",
"South Sudan",
"Spain",
"Sri Lanka",
"Sudan",
"Suriname",
"Sweden",
"Switzerland",
"Syria",
"Tajikistan",
"Thailand",
"Timor-Leste",
"Togo",
"Tonga",
"Trinidad and Tobago",
"Tunisia",
"Turkey",
"Turkmenistan",
"Tuvalu",
"Uganda",
"Ukraine",
"United Arab Emirates",
"United Kingdom",
"United Republic of Tanzania",
"United States of America",
"Uruguay",
"Uzbekistan",
"Vanuatu",
"Venezuela (Bolivarian Republic of)",
"Viet Nam",
"Yemen",
"Zambia",
"Zimbabwe"]